# PageRank Implementation

In [42]:
import numpy as np
from collections import defaultdict
from scipy import sparse

small = "dataset/CA-GrQc-1.txt"
large = "dataset/com-dblp.ungraph.txt"

In [41]:
class PageRank:
    def __init__(self, damping=0.85, max_iter=500, tol=1e-6):
        self.damping = damping
        self.max_iter = max_iter
        self.tol = tol
        self.pagerank = None
        self.pages = None
    
    def build_transition_matrix(self, graph):
        self.pages = sorted(graph.keys())
        n = len(self.pages)
        M = np.zeros((n, n))
        page_index = {page: i for i, page in enumerate(self.pages)}
        
        for from_page, to_pages in graph.items():
            if len(to_pages) == 0:
                M[:, page_index[from_page]] = 1/n
            else:
                prob = 1 / len(to_pages)
                for to_page in to_pages:
                    M[page_index[to_page], page_index[from_page]] = prob
        return M
    
    def build_pagerank_matrix(self, M):
        n = M.shape[0]
        dangling_weights = np.where(M.sum(axis=0) == 0, 1/n, 0)
        P = self.damping * M + self.damping * dangling_weights + (1 - self.damping) / n
        return P
    
    def power_iteration(self, P):
        n = P.shape[0]
        v = np.ones(n) / n
        
        for _ in range(self.max_iter):
            v_new = P @ v
            if np.linalg.norm(v_new - v) < self.tol:
                break
            v = v_new
        return v
    
    def rank(self, graph):
        M = self.build_transition_matrix(graph)
        P = self.build_pagerank_matrix(M)
        self.pagerank = self.power_iteration(P)
        return {page: score for page, score in zip(self.pages, self.pagerank)}

In [38]:
small_web_graph = defaultdict(lambda: [])

with open(small, "r") as f:
    small_relations = f.readlines()[1:]

for r in small_relations:
    u, v = map(int, r.strip().split("\t"))
    small_web_graph[u].append(v)



pagerank = PageRank()
results = pagerank.rank(small_web_graph)
print("PageRank Results:")
for page, score in sorted(results.items(), key=lambda x: -x[1]):
    print(f"{page}: {score:.4f}")

PageRank Results:
14265: 0.0014
13801: 0.0013
13929: 0.0013
21281: 0.0012
9572: 0.0012
2710: 0.0011
22691: 0.0011
21012: 0.0011
7689: 0.0011
6264: 0.0011
12365: 0.0011
449: 0.0010
4952: 0.0010
9017: 0.0010
9124: 0.0010
5052: 0.0009
6610: 0.0009
10762: 0.0009
17655: 0.0009
7307: 0.0009
1488: 0.0009
14599: 0.0009
19865: 0.0009
23038: 0.0009
15108: 0.0009
19423: 0.0009
24924: 0.0009
13142: 0.0008
21508: 0.0008
9785: 0.0008
7007: 0.0008
1217: 0.0008
5901: 0.0008
14746: 0.0008
24559: 0.0008
18866: 0.0008
2741: 0.0008
2654: 0.0008
23614: 0.0008
20765: 0.0008
5346: 0.0008
20373: 0.0008
18208: 0.0008
543: 0.0008
9710: 0.0008
10711: 0.0008
1000: 0.0008
13276: 0.0008
15003: 0.0008
14924: 0.0008
3501: 0.0007
14807: 0.0007
2042: 0.0007
24057: 0.0007
3651: 0.0007
4164: 0.0007
1653: 0.0007
12927: 0.0007
15244: 0.0007
17075: 0.0007
20511: 0.0007
15066: 0.0007
23382: 0.0007
12842: 0.0007
20478: 0.0007
13556: 0.0007
7350: 0.0007
13096: 0.0007
2072: 0.0007
12781: 0.0007
12545: 0.0007
4550: 0.0007
16469:

In [39]:
class OptimizedPageRank:
    def __init__(self, damping=0.85, max_iter=100, tol=1e-6, chunk_size=10000):
        self.damping = damping
        self.max_iter = max_iter
        self.tol = tol
        self.chunk_size = chunk_size
        self.pagerank = None
        self.pages = None
        self.page_index = None
    
    def _process_graph_chunks(self, graph):
        all_pages = sorted(set(graph.keys()) | {p for targets in graph.values() for p in targets})
        self.pages = all_pages
        self.page_index = {page: i for i, page in enumerate(all_pages)}
        n = len(all_pages)
        
        rows, cols, data = [], [], []
        dangling_nodes = []
        
        for i in range(0, n, self.chunk_size):
            chunk_pages = all_pages[i:i+self.chunk_size]
            
            for from_page in chunk_pages:
                to_pages = graph[from_page]
                from_idx = self.page_index[from_page]
                
                if not to_pages: 
                    dangling_nodes.append(from_idx)
                else:
                    prob = 1 / len(to_pages)
                    for to_page in to_pages:
                        rows.append(self.page_index[to_page])
                        cols.append(from_idx)
                        data.append(prob)
        
        return rows, cols, data, dangling_nodes, n
    
    def build_transition_matrix(self, graph):
        rows, cols, data, dangling_nodes, n = self._process_graph_chunks(graph)
        
        M = sparse.coo_matrix((data, (rows, cols)), shape=(n, n)).tocsr()
        self.dangling_nodes = dangling_nodes
        
        dangling_vec = np.zeros(n)
        dangling_vec[dangling_nodes] = 1/n
        self.dangling_vec = sparse.csr_matrix(dangling_vec)
        
        return M
    
    def build_pagerank_matrix(self, M):
        n = M.shape[0]
        teleport = (1 - self.damping) / n
        P = self.damping * M

        if self.dangling_nodes:
            dangling_contrib = np.zeros(n)
            dangling_contrib[self.dangling_nodes] = 1.0
            dangling_contrib = self.damping * dangling_contrib / n
            self.dangling_contrib = dangling_contrib
        else:
            self.dangling_contrib = np.zeros(n)

        self.teleport = np.ones(n) * teleport
        return P

    
    def power_iteration(self, P):
        n = P.shape[0]
        v = np.ones(n) / n
        squared_tol = self.tol ** 2
        
        for _ in range(self.max_iter):
            v_new = P.dot(v) + np.sum(v[self.dangling_nodes]) * self.dangling_contrib + self.teleport

            
            diff = v_new - v
            if np.dot(diff, diff) < squared_tol:
                break
                
            v = v_new
            
        return v
    
    def rank(self, graph):
        M = self.build_transition_matrix(graph)
        
        P = self.build_pagerank_matrix(M)
        
        self.pagerank = self.power_iteration(P)
        
        return ((page, self.pagerank[self.page_index[page]]) for page in self.pages)

    def top_n(self, n=10):
        if self.pagerank is None:
            raise ValueError("Must call rank() first")
            
        top_indices = np.argpartition(-self.pagerank, n)[:n]
        top_indices_sorted = top_indices[np.argsort(-self.pagerank[top_indices])]
        
        return [(self.pages[i], self.pagerank[i]) for i in top_indices_sorted]

def generate_large_graph(num_nodes=1000000, avg_links=10):
    graph = defaultdict(list)
    nodes = [f"node_{i}" for i in range(num_nodes)]
    
    for i in range(num_nodes):
        num_links = avg_links + (hash(nodes[i]) % 7 - 3)
        links = [nodes[hash(f"{i}_{j}") % num_nodes] for j in range(num_links)]
        graph[nodes[i]] = links
        
    return graph

web_graph = defaultdict(lambda: [])

with open(small, "r") as f:
    small_relations = f.readlines()[1:]

for r in small_relations:
    u, v = map(int, r.strip().split("\t"))
    web_graph[u].append(v)

pr = OptimizedPageRank()
results = pr.rank(web_graph)

print("Top Pages by PageRank:")
for page, score in pr.top_n(3):
    print(f"{page}: {score:.6f}")

Top Pages by PageRank:
14265: 0.001443
13801: 0.001341
13929: 0.001305
